Utilizing GPU  

In [0]:
import torch

if torch.cuda.is_available():    
  
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
!nvidia-smi

Thu May  7 22:03:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     7W /  75W |     10MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

all requirements

In [0]:
import glob
import pandas as pd
import os
import numpy as np
!pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 39.8MB/s 
     |████████████████████████████████| 3.8MB 39.1MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b933faadca4e9973e47398d1d3eee0a6fc22cf335e247a920a36770886d21830
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
## NADI_SHARED_TASK cloning

In [0]:
cd NADI_Shared_Task/

/content/NADI_Shared_Task


In [0]:
!pip install -r requirements.txt

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 112kB 8.7MB/s 
  Created wheel for PyArabic: filename=PyArabic-0.6.7-cp36-none-any.whl size=108603 sha256=5bed89b1102664f4e9847d0104ee6997efb7deef2bea982d90d838c4a6f8ca44
  Stored in directory: /root/.cache/pip/wheels/c6/50/b1/4df7f705f36e91360ab04416dbf1017084698d30a7a3645b5e
Successfully built PyArabic


In [0]:
from transformers import AutoTokenizer
from utils.preprocess_arabert import never_split_tokens,preprocess

tokenizer = AutoTokenizer.from_pretrained(
    "aubmindlab/bert-base-arabert",
    do_lower_case=False,
    do_basic_tokenize=True,
    never_split=never_split_tokens)



reading tweets and cleaning them using arabert preprocessor

In [0]:
def read_tweets_unlabeled(path):
  filenames = glob.glob(os.path.join(path, "*.csv"))

  li = []

  for file in filenames:
    df = pd.read_csv(file)
    li.append(df)

  frame = pd.concat(li, axis=0, ignore_index=True)
  del li
  frame = frame.dropna()
  return frame

In [0]:
data = read_tweets_unlabeled('../drive/My Drive/NADI_release/Cleaned')

In [0]:
data = data[1000000:1500000]

In [0]:
data

,tweets
1062681,ليتك تحّس بحالتي لا فقدتكِ شلونكم يأهل تويتر
1062682,اغتصبوا الثورات وصنعوا ديمقراطية وهمية حسبوا ا...
1062683,أعيذك بكلمات الله التامة من كل شيطان وهامة وكل...
1062684,جميع ماقدمّه الدون رونالدو في نهائي دوري ابطال...
1062685,خلاصه القول ولله
...,...
1604126,دعم خاص قدرك كبير وخوتك ترفع الراس فوق الغلا ت...
1604127,عدة مقترحات قدمت لهم ولكن لم يقدم احدا شيء يرض...
1604128,كلنا ثقه في الهلال نصيحة وجه السعد الامير نواف...
1604130,التی أنعمت


In [0]:
from py4j.java_gateway import JavaGateway

In [0]:
gateway = JavaGateway.launch_gateway(classpath='./FarasaSegmenterJar.jar')

In [0]:
farasa = gateway.jvm.com.qcri.farasa.segmenter.Farasa()

In [0]:
data['tweets'] = data['tweets'].apply(lambda x : preprocess(x, do_farasa_tokenization=True , farasa=farasa))

changing path to save it in clean_data directory

In [0]:
cd ..

/content


In [0]:
cd 'drive/My Drive/NADI_release/transformers/clean_data'

/content/drive/My Drive/NADI_release/transformers/clean_data


In [0]:
data.to_csv('1500000.csv', index=False) 